In [1]:
import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries
#from shared_lib import utils, vocabulary, tf_embed_viz



### Parse data from ../data/processed

In [2]:
def get_vocab2(raw):
    tokens = nltk.word_tokenize(raw) #.decode("utf8"))
    words = [w.lower() for w in tokens]
    vocab = sorted(set(words))
    sents = nltk.sent_tokenize(raw.lower())
    return vocab, sents

    
def get_vocab(filename):
    raw = open(filename).read().decode("UTF-8")
    return get_vocab2(raw)

dir = '../data/processed/'
pres_dict = {}

def add_to_dict(president, vocab, sents):
    vocab_, sents_ = pres_dict.get(president, (None, None))
    if (vocab_ == None):
        pres_dict[president] = (vocab, sents)
    else:
        vocab_ += vocab
        sents_ += sents
        pres_dict[president] = (vocab_, sents_)

def print_dict(pres_dict):        
    for key in pres_dict.keys():
        vocab, sents = pres_dict.get(key, (None, None))
        print "%s: vocab length %s, sents length %s" % (key, len(vocab), len(sents))
        #print "Samples: ", sents[-4:]
    
for filename in os.listdir(dir):
    arr = filename.split("_")
    president = arr[0]
    #vocab, sents = get_vocab(dir + filename)
    #add_to_dict(president, vocab, sents)

    try:
        vocab, sents = get_vocab(dir + filename)
        add_to_dict(president, vocab, sents)
    except UnicodeDecodeError as err:
        print filename, ":", err
        

        
print_dict(pres_dict)

Lincoln: vocab length 15975, sents length 2102
Trump: vocab length 544, sents length 90
Obama: vocab length 15305, sents length 3106


### Parse json data from ../data/unprocessed

In [4]:
from bs4 import BeautifulSoup
import re


dir = '../data/unprocessed/'

for json_file in os.listdir(dir):
    json_data=open(dir + json_file)
    data = json.load(json_data)
    json_data.close()
    attrName = 'debate' if 'Debate' in json_file else 'speeches'
    for data2 in data[attrName]:
        raw = BeautifulSoup(data2['text']).get_text()
        # Remove []
        raw = re.sub(' \[.*?\]',' ', raw, flags=re.DOTALL)
        # Remove ()
        raw = re.sub(' \(.*?\)',' ', raw, flags=re.DOTALL)
        if attrName == 'speeches':
            # print data2['date'], data2['name'], data2['speaker']#, debate['text'], debate['url'], 
            # data2['text'] has a lot of htmtl tags in there. We still need to parse it            
            # Removing the questions
            raw = re.sub(' Q\..*? The President\.','',raw, flags=re.DOTALL)
            
            #print data2['text'][0:200]
            #print data2['url']
            #print raw
            #break
            
            vocab, sents = get_vocab2(raw)
            arr = data2['speaker'].split(' ')
            president = arr[len(arr)-1]
            #print "*** %s: %s: new vocab length %s, sents length %s" % (data2['speaker'], president, len(vocab), len(sents))
            # Remove the last few sentences because they are not part of the speech
            #print sents[-10:]
            sents = sents[:len(sents)-10]
            #print sents[-2:]
            #print len(sents)
            add_to_dict(president, vocab, sents)

        ########################################################
        # TODO: extract debate data for TRUMP, OBAMA
        #elif ('OBAMA' in raw): #('TRUMP' in raw or 'OBAMA' in raw)):
        #    print data2['name'] #, raw[0:3000]


print_dict(pres_dict)


Lincoln: vocab length 15975, sents length 2102
Trump: vocab length 22740, sents length 7224
Bush: vocab length 2644, sents length 802
Obama: vocab length 460563, sents length 89546
